In [1]:
# Simple mnist convnet. (99% acc on MNist)
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Prepare the data.
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
# Build the model.
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

2021-08-13 18:35:53.040709: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Train the model.
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

2021-08-13 18:36:11.701599: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
422/422 [==============================] - 32s 73ms/step - loss: 0.3528 - accuracy: 0.8939 - val_loss: 0.0793 - val_accuracy: 0.9787
Epoch 2/15
422/422 [==============================] - 31s 73ms/step - loss: 0.1136 - accuracy: 0.9653 - val_loss: 0.0604 - val_accuracy: 0.9832
Epoch 3/15
422/422 [==============================] - 30s 71ms/step - loss: 0.0856 - accuracy: 0.9738 - val_loss: 0.0530 - val_accuracy: 0.9863
Epoch 4/15
422/422 [==============================] - 30s 70ms/step - loss: 0.0726 - accuracy: 0.9779 - val_loss: 0.0448 - val_accuracy: 0.9885
Epoch 5/15
422/422 [==============================] - 30s 72ms/step - loss: 0.0643 - accuracy: 0.9798 - val_loss: 0.0390 - val_accuracy: 0.9893
Epoch 6/15
422/422 [==============================] - 30s 71ms/step - loss: 0.0583 - accuracy: 0.9816 - val_loss: 0.0384 - val_accuracy: 0.9905
Epoch 7/15
422/422 [==============================] - 30s 71ms/step - loss: 0.0536 - accuracy: 0.9827 - val_loss: 0.0357 - val_accuracy:

In [5]:
# Evaluate the trained model.
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.025723908096551895
Test accuracy: 0.9914000034332275


In [6]:
# Save the model. (HDF5JSON)
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.hdf5")

In [8]:
import tensorflow as tf
# Get TFLite from a simple model. (https://www.tensorflow.org/lite/convert)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model. (TFLite)
model_path = 'simple_mnist.tflite'
with open(model_path, 'wb') as f:
    f.write(tflite_model)

2021-08-13 18:46:14.180819: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/1v/gx3qc3cn3rgbn3yrq2_l_xq40000gn/T/tmplwp7na2w/assets


2021-08-13 18:46:14.894629: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-08-13 18:46:14.895137: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-08-13 18:46:14.909456: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.003ms.

2021-08-13 18:46:15.018421: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-08-13 18:46:15.018453: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
2021-08-13 18:46:15.090095: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:210] disabling MLIR crash reproducer, set env var `